In [1]:
#!pip install tokenizers

In [2]:
import os
import sys

# Add paths for custom transformers
while any(marker in os.getcwd() for marker in ('exercises', 'notebooks', 'students', 'research', 'projects')):
    os.chdir("..")
sys.path.append('.')

In [ ]:
import kagglehub
from dotenv import load_dotenv
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from src.custom_transformers import (
    CustomOneHotEncoder,
    CustomStandardScaler
)
import projects.proj_3_team_5.src.custom_transformers as project_transformers
from projects.proj_3_team_5.src.custom_transformers import CustomTokenizerVectorizer, ToDataFrameFromColumnTransformer



In [ ]:
env_path = 'projects/proj_3_team_5/.env'
load_dotenv(env_path)

In [ ]:
# Download latest version
path = kagglehub.dataset_download("ruthgn/beer-profile-and-ratings-data-set")

print("Path to dataset files:", path)

In [6]:
df_raw_path = os.getenv('RAW_DATA_DIR')
df_preprocessed_path = os.getenv('PREPROCESSED_DATA_DIR')

In [ ]:
# Set the correct path to your raw data file if not already set
if not df_raw_path:
	# Example: update the filename as needed
	df_raw_path = os.path.join(path, "beer_profile_and_ratings.csv")

df_raw = pd.read_csv(df_raw_path)
df_raw.info()

In [ ]:
df_raw.sample(5)

In [ ]:
print("\nMissing Values:")
missing = df_raw.isnull().sum()
print(missing[missing > 0].sort_values(ascending=False))

# Add empty cells check
empty_cells = df_raw.eq('').sum()
print("\nEmpty String Values:")
print(empty_cells[empty_cells > 0].sort_values(ascending=False))

In [ ]:
num_dupes = df_raw.duplicated().sum()
print(f"\nNumber of duplicate rows in train data: {num_dupes}")

In [ ]:
# Check cardinality of categorical columns
categorical_cols = df_raw.select_dtypes(include=['object']).columns
cardinality = {col: df_raw[col].nunique() for col in categorical_cols}
print("Number of unique values in categorical columns:")
for col, count in sorted(cardinality.items(), key=lambda x: x[1], reverse=True):
    print(f"{col}: {count}")

In [ ]:
# Check if all descriptions start with "Notes:"
all_start_with_notes = df_raw['Description'].str.startswith('Notes:').all()
print(f"All descriptions start with 'Notes:': {all_start_with_notes}")

# Show some examples if not all start with "Notes:"
if not all_start_with_notes:
    print("\nExamples of descriptions that don't start with 'Notes:':")
    non_notes = df_raw[~df_raw['Description'].str.startswith('Notes:')]['Description'].head()
    print(non_notes)


In [ ]:
# Get numeric columns
numeric_cols = df_raw.select_dtypes(include=['int64', 'float64']).columns.tolist()
print("Numeric columns:")
print(numeric_cols)


In [14]:
cleaning_pipeline = make_pipeline(
    project_transformers.TextCleaner(
        column='Description',
        patterns_to_remove=['^Notes:', 'error entering this description']
    ),
    project_transformers.CustomOutlierRemover(
        columns=numeric_cols,
        threshold=3
    )
)

In [ ]:
df_cleaned = cleaning_pipeline.fit_transform(df_raw)

print("Original shape:", df_raw.shape)
print("Processed shape:", df_cleaned.shape)

In [ ]:
cleaning_pipeline

In [ ]:
# Count empty cells in Description column
empty_descriptions = df_cleaned['Description'].isna().sum() + (df_cleaned['Description'] == '').sum()
print(f"Number of empty cells in Description: {empty_descriptions}")


In [ ]:
df_cleaned.sample(5)

In [19]:
df_cleaned_path = os.getenv('CLEANED_DATA_DIR')

In [ ]:
df_cleaned.to_csv(df_cleaned_path, index=False)

In [21]:
transformers=ColumnTransformer([
            ('name_tok', CustomTokenizerVectorizer(column='Name', vocab_size=300), ['Name']),
            ('desc_tok', CustomTokenizerVectorizer(column='Description', vocab_size=1000), ['Description']),
            ('beer_tok', CustomTokenizerVectorizer(column='Beer Name (Full)', vocab_size=500), ['Beer Name (Full)']),
        ],
        remainder='passthrough', sparse_threshold=0, 
        verbose_feature_names_out=False)


In [22]:
final_pipeline = make_pipeline(
    transformers,
    ToDataFrameFromColumnTransformer(transformers),
    CustomOneHotEncoder(
        columns=['Style', 'Brewery'],
    ),
    CustomStandardScaler(
        columns=numeric_cols
    )
)

In [ ]:
df_preprocessed = final_pipeline.fit_transform(df_cleaned)

In [ ]:
final_pipeline

In [ ]:
df_preprocessed.shape

In [ ]:
df_preprocessed.sample(5)

In [ ]:
df_preprocessed.to_csv(df_preprocessed_path, index=False)